In [1]:
!pip install duckduckgo-search
!pip install wikipedia
!pip install langchain
!pip install langchainhub
!pip install langchain_openai

  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
Using cached click-8.1.7-py3-none-any.whl (97 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached charset_normalizer-3.3.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (33 kB)
  Using cached idna-3.8-py3-none-any.whl.metadata (9.9 kB)
  Using cached urllib3-2.2.2-py3-none-any.whl.metadata (6.4 kB)
  Using cached certifi-2024.7.4-py3-none-any.whl.metadata (2.2 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached certifi-2024.7.4-py3-none-any.whl (162 kB)
Using cached charset_normalizer-3.3.2-cp312-cp312-macosx_11_0_arm64.whl (119 kB)
Using cached idna-3.8-py3-none-any.whl (66 kB)
Using cached urllib3-2.2.2-py3-none-any.whl (121 kB)
  Created wheel for wikipedia: filenam

In [2]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor
from langchain.schema import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

In [8]:
!pip install python-dotenv

  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)


In [10]:
import os
from dotenv import load_dotenv

load_dotenv()

MY_ENV_VAR = os.getenv('OPENAI_API_KEY')

In [11]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
messages = [
    SystemMessage(
        content="A User will input in a year and you will get the baseball world series champion"
    ),
    HumanMessage(
        content="2023"
    ),
]
llm.invoke(messages)

AIMessage(content='The 2023 World Series champion was the Atlanta Braves.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 29, 'total_tokens': 41}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-891eec42-0351-409c-91b0-d6c63fbb1abd-0', usage_metadata={'input_tokens': 29, 'output_tokens': 12, 'total_tokens': 41})

In [19]:
!pip install --upgrade --quiet  wikipedia
!pip install --quiet langchain-community langchain-core

In [20]:
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [35]:
from langchain_core.prompts import PromptTemplate

prompt_text = """
system
You are a helpful assistant

placeholder
chat_history

human
{input}

placeholder
{agent_scratchpad}
"""

prompt = PromptTemplate.from_template(prompt_text)

In [29]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=500)
tools = [WikipediaQueryRun(api_wrapper=api_wrapper)]

In [36]:
agent = create_openai_functions_agent(llm, tools, prompt)

In [37]:
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [40]:
agent_executor.invoke({"input": "Who won the 2023 world series?"})

{'input': 'Who won the 2023 world series?',
 'output': 'The Texas Rangers won the 2023 World Series, defeating the Arizona Diamondbacks in five games.'}

In [41]:
from langchain import LLMMathChain
from langchain.agents import Tool, initialize_agent
from langchain.tools import DuckDuckGoSearchRun

In [44]:
!pip install numexpr

In [45]:
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
math_tool = Tool.from_function(
        func=llm_math_chain.run,
        name="Calculator",
        description="Useful for when you need to answer questions about math. This tool is only for math questions and nothing else. Only input math expressions.",
    )

In [46]:
search = DuckDuckGoSearchRun()

In [47]:
tools2 = [math_tool, search]

In [49]:
agent2 = create_openai_functions_agent(llm= llm, tools = tools2, prompt=prompt)

In [50]:
agent_executor2 = AgentExecutor(agent=agent2, tools=tools2, verbose=True)

In [51]:
agent_executor2.invoke({"input": "Who won the 2021 world series and how many years was it since their last world series win"})



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `{'query': '2021 World Series winner'}`


The World Series is the annual championship series of Major League Baseball (MLB) and concludes the MLB postseason.First played in 1903, [1] the World Series championship is a best-of-seven playoff and is a contest between the champions of baseball's National League (NL) and American League (AL). [2] Often referred to as the "Fall Classic", [3] the modern World Series has been played every ... The Texas Rangers claimed their first World Series title in team history, beating the Arizona Diamondbacks in Game 5. ... manager Chris Young after Texas lost 102 games in 2021 and went 68-94 last ... World Series winners list. Here's a breakdown of each season's MLB World Series matchup and winner: ... 2022: Houston Astros def. Philadelphia Phillies; 2021: Atlanta Braves def. Houston Astros ... Here's a quick look at every World Series winner in MLB history. World Series. Winning

{'input': 'Who won the 2021 world series and how many years was it since their last world series win',
 'output': 'The Atlanta Braves won the 2021 World Series. It had been 26 years since their last World Series win in 1995.'}